In [1]:
from premise import *
import bw2data, bw2io, bw2calc
bw2data.projects.set_current("ecoinvent-3.12-cutoff")
#bw2data.projects.set_current("ecoinvent-3.10-cutoff")
#bw2data.projects.set_current("ei-3.11-bw25")

/opt/homebrew/Caskroom/miniforge/base/envs/premise/lib/python3.11/site-packages/scikits/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


15:39:33+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
bw2data.databases

Databases dictionary with 23 objects, including:
	biofuels test 2
	biosphere3
	ecoinvent-3.12-cutoff
	ei_cutoff_3.10.1_gcam_SSP2-Base_2050 2026-02-04
	ei_cutoff_3.10.1_image_SSP1-L_2025 2026-02-04
	ei_cutoff_3.10.1_message_SSP1-L_2050 2026-02-04
	ei_cutoff_3.10.1_remind_SSP2-PkBudg650_2050 2026-02-04
	ei_cutoff_3.10.1_tiam-ucl_SSP2-RCP19_2050 2026-02-04
	ei_cutoff_3.12_message_SSP1-L_2050 2026-02-04
	ei_cutoff_3.12_message_SSP1-L_2050 2026-02-14
Use `list(this object)` to get the complete list.

In [3]:
#clear_inventory_cache()

In [4]:
scenarios = [
    #{"model": "remind", "pathway": "SSP1-NPi", "year": 2050,},
    #{"model": "remind", "pathway": "SSP1-PkBudg650", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg1000", "year": 2025,},
    
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2050,},
    #{"model": "remind", "pathway": "SSP2-PkBudg1000", "year": 2025,},
    
    #{"model": "remind", "pathway": "SSP3-rollBack", "year": 2025,},

    #{"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2050,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2025,},

    #{"model": "image", "pathway": "SSP1-L", "year": 2050,},
    #{"model": "image", "pathway": "SSP1-Ma", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-VLLO", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-M", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-VLHO", "year": 2025,},
    #{"model": "image", "pathway": "SSP5-H", "year": 2025,},
    #{"model": "image", "pathway": "SSP3-H", "year": 2025,},

    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2050,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP26", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP45", "year": 2025,},

    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2050,},

    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2050,},

    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2050,},

    #{"model": "gcam", "pathway": "SSP2-Base", "year": 2050,},
    #{"model": "gcam", "pathway": "SSP2-RCP26", "year": 2050,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2050}

    {"model": "message", "pathway": "SSP1-L", "year": 2050},
    #{"model": "message", "pathway": "SSP1-VL", "year": 2050},
    #{"model": "message", "pathway": "SSP2-L", "year": 2050},
    #{"model": "message", "pathway": "SSP2-LO", "year": 2050},
    #{"model": "message", "pathway": "SSP2-M", "year": 2050},
    #{"model": "message", "pathway": "SSP2-ML", "year": 2050},
    #{"model": "message", "pathway": "SSP2-VL", "year": 2050},
    #{"model": "message", "pathway": "SSP3-H", "year": 2050},
    #{"model": "message", "pathway": "SSP4-LO", "year": 2050},
    #{"model": "message", "pathway": "SSP5-H", "year": 2050},
    #{"model": "message", "pathway": "SSP5-LO", "year": 2050},
    
]


ndb = NewDatabase(
        scenarios = scenarios,
        source_db="ecoinvent-3.12-cutoff",
        source_version="3.12",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        biosphere_name="biosphere3",
)


premise v.(2, 3, 7)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your Brightway project:                                   |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [5]:
ndb.update()

Processing scenarios for all sectors:   0%|     | 0/1 [00:00<?, ?it/s]

No EWR energy mix data available -- skipping
No car fleet scenario data available -- skipping
No two-wheeler fleet scenario data available -- skipping
No truck fleet scenario data available -- skipping
No ship fleet scenario data available -- skipping
No bus fleet scenario data available -- skipping
No train fleet scenario data available -- skipping


Processing scenarios for all sectors: 100%|█| 1/1 [01:44<00:00, 104.81

Done!



In [6]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
Database ei_cutoff_3.12_message_SSP1-L_2050 2026-02-14 already exists: it will be overwritten.
15:42:40+0100 [info     ] Vacuuming database            
15:45:59+0100 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████| 40959/40959 [01:05<00:00, 626.33it/s]

15:47:19+0100 [info     ] Vacuuming database            


Created database: ei_cutoff_3.12_message_SSP1-L_2050 2026-02-14
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [7]:
from premise.utils import load_database

db = load_database(ndb.scenarios[0], ndb.database)["database"]

In [8]:
for ds in db:
    if ds["name"].startswith("hardwood forestry, birch, sustainable forest management"):
        for e in ds["exchanges"]:
            print(e["name"], "|", e.get("product"), "|", e["amount"], "|", e["unit"], "|", e.get("location"))
        print()

hardwood forestry, birch, sustainable forest management | sawlog and veneer log, hardwood, with bark, green, measured as green volume under bark | 1.0 | cubic meter | SE
market for diesel, burned in building machine | diesel, burned in building machine | 7.878255 | megajoule | GLO
forwarding, forwarder | forwarding, forwarder | 0.048244856 | hour | RoE
market for gravel, crushed | gravel, crushed | 8.1872425 | kilogram | RoW
harvesting, forestry harvester | harvesting, forestry harvester | 0.0979 | hour | RoE
power sawing, without catalytic converter | power sawing, without catalytic converter | 0.12415918 | hour | RER
skidding, skidder | skidding, skidder | 0.00134 | hour | RER
Carbon dioxide, in air | None | 1129.6279 | kilogram | None
Energy, gross calorific value, in biomass | None | 12223.431 | megajoule | None
Occupation, forest, intensive | None | 1229.8889 | square meter-year | None
Occupation, traffic area, rail/road embankment | None | 4.6790123 | square meter-year | None
Tra

In [9]:
len(db)

29384